# Métodos sem malha
Este trabaho contará com a implementação de 03 métodos sem malha para estruturas elásticas e viscoelásticas. Os métodos a serem implementados serão o da Colocação, Galerkin e Petrov-Galerkin. Todos utilizarão o Método dos Mínimos Quadrados Móveis para interpolação.

# Dependências
A seguir encontram-se as bibliotecas utilizadas no presente trabalho para jupyter.

In [13]:
%matplotlib notebook

import matplotlib
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from numpy import linalg as la
import matplotlib.pyplot as plt
import pprint as pp

# Dados
Abaixo encontram-se os dados utilizados nos problemas abaixo.
 
  - interpolação: $z = 3+2x+5x^2+4xy$

In [4]:
interpol_data = [
    [-1,-1,10],
    [0,-1,3],
    [1,-1,6],
    [-1,0,6],
    [0,0,3],
    [1,0,10],
    [-1,1,2],
    [0,1,3],
    [1,1,14]
]
interpol_base = ["1","x","x**2","y","y**2","x*y"];




# Definição Função Peso

Criar a função gaussiana com raio e Função Spline, verificando qual se comporta melhor na modelagem.

$\textbf{Função Gaussiana com raio:}$

$
w(\vec d) = 
\begin{cases}
\dfrac{{e^{-({\frac{||\vec d||}c})^2}} - {e^{-({\frac r c})^2}}}{1-{e^{-({\frac{r}c})^2}}}, \ d < r
\\\\
0, \ d > r
\end{cases}
$

$\textbf{Função Spline:}$

$
w(\vec d) = 
\begin{cases}
\ {1- 6{({\frac{||\vec d||}r})^2} + 8{({\frac{||\vec d||}r})^3} - 3{({\frac{||\vec d||}r})^4}}, \ d < r
\\\\
0, \ d > r
\end{cases}
$

$\textbf{Obs:}$ 
- dist: $\textbf{x} $ - $\textbf{x$_{j}$} $ = $\vec d $
    
- r: raio limite do subdominio (suporte)
    
    
 

In [14]:
def gaussian_with_radius (dist,r):
    c = 100
    exp1 = np.exp(-(la.norm(dist)/c)**2) #Exponencial 2
    exp2 = np.exp(-(r/c)**2) #Exponencial 1
    weight = (exp1 - exp2)/(1 - exp2)
    
    return weight if (la.norm(dist)<=r) else 0 


def spline (dist,r):
    div = la.norm(dist)/r
    weight = 1 - 6*(div**2)+8*(div**3)-3*((div**4))
    
    return weight if (la.norm(dist) <=r) else 0


# Matriz Peso (W)

In [ ]:
def W (data,point,r):
    W = []
    